In [ ]:
# Ausführen um die Verbindung zur Datenbank aufzubauen

import sqlite3
import pandas as pd
import urllib.request

urllib.request.urlretrieve("https://github.com/tschuegge/SqlKurs/raw/master/database.db", "database.db")
conn = sqlite3.connect("database.db")

def query(query):
  return pd.read_sql_query(query, conn)

print("😊 Verbunden mit Sqlite " + sqlite3.sqlite_version)

In [ ]:
# Aufgabe 1.5.1
query("""

SELECT *
FROM hersteller h
WHERE h.id NOT IN (SELECT a.herstellerid
                   FROM artikel a
                     INNER JOIN bestellungposition p ON p.artikelid = a.id
                  )

""")

In [ ]:
# Aufgabe 1.5.2
query("""

SELECT *
FROM artikel a
  INNER JOIN bestellungposition bp ON a.id = bp.bestellungkopfid
  INNER JOIN bestellungkopf bk ON bk.id = bp.bestellungkopfid
  INNER JOIN kunde kd ON bk.kundeid = kd.id
WHERE kd.ort IN (SELECT ort FROM mitarbeiter)

""")

In [ ]:
# Aufgabe 1.5.3
query("""



""")